In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from colorama import Fore, Style

In [4]:
clust = pd.read_csv('data/clustering.csv')
mnist = pd.read_csv('data/MNIST_dataset.csv')

# 1)

No hay mucho que ver en los datos, se puede interpretear bastante claramente a simple vista. <br>
Saco la columna redundante de los indices de fila, y g rafico A vs B a ver si me dice algo (en MNIST no hago nada en principio)

In [ ]:
# Base Process
clust = clust.drop("Unnamed: 0", axis=1)

In [ ]:
# Print clust preview
print(Fore.CYAN + "Clust DataFrame Preview:" + Style.RESET_ALL)
print(clust.head(n=15))

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=clust, x='A', y='B', alpha=0.5)
plt.title('Scatter plot of A vs B')
plt.xlabel('A')
plt.ylabel('B')
plt.show()